In [48]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
from os import listdir
from os.path import isfile, join
import numpy as np  
import librosa
import librosa.display
import pandas as pd 
import matplotlib.pyplot as plt

In [49]:
mypath = os.path.join('RespiratorySoundDatabase','Respiratory_Sound_Database','Respiratory_Sound_Database','audio_and_txt_files')
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))] 


In [50]:
p_id_in_file = [] # patient IDs corresponding to each file
for name in filenames:
    p_id_in_file.append(int(name[:3]))

p_id_in_file = np.array(p_id_in_file)

In [76]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name):
    """
    This function takes in the path for an audio file as a string, loads it, and returns the MFCC
    of the audio"""
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', duration=20.0) 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    return mfccs
extract_features('RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Al_sc_Meditron.wav')

Error encountered while parsing file:  RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Al_sc_Meditron.wav


In [40]:
filepaths = [join(mypath, f) for f in filenames] # full paths of files

# for f in filepaths:
#     print(f)

In [41]:
p_diag = pd.read_csv("./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv",header=None) # patient diagnosis file
# print(p_diag)

In [44]:
labels = np.array([p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file]) # labels for audio files
# print(labels)

In [45]:
features = [] 

# Iterate through each sound file and extract the features
for file_name in filepaths:
    data = extract_features(file_name)
    features.append(data)

print('Finished feature extraction from ', len(features), ' files')
features = np.array(features)

Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\101_1b1_Al_sc_Meditron.wav
Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\101_1b1_Pr_sc_Meditron.wav
Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\102_1b1_Ar_sc_Meditron.wav
Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\103_2b2_Ar_mc_LittC2SE.wav
Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Al_sc_Litt3200.wav
Error encountered while parsing file:  RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Ar_sc_Litt3200.wav
Error enco